<a href="https://colab.research.google.com/github/mdehghani86/AppliedGenAI/blob/main/LangGraph_Tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LangGraph Lab : Part 1 🚀**
- Prof. Dehghani
## **Building More Controllable LLM Agents with LangGraph**

## **Introduction**
LangGraph is a framework designed for building **agent and multi-agent applications** with structured control. While large language models (LLMs) are powerful, they often require **more precise workflows** to ensure reliability.

Many real-world applications need agents to follow **specific steps**, such as always calling a certain tool first or adjusting their prompts based on the current state. Traditional agent frameworks may not provide enough control for these scenarios. LangGraph introduces a **graph-based approach** that allows developers to define structured workflows while still benefiting from LLM flexibility.

This lab is adapted from [LangChain Academy's Intro to LangGraph](https://academy.langchain.com/courses/intro-to-langgraph), which provides additional details and use cases.

## **Lab Objectives**
By the end of this lab, the following concepts will be covered:
- The role of **graphs** in LLM-based agent workflows
- How to define **nodes** (decision points) and **edges** (paths) in LangGraph
- Implementing an **agent with controlled decision-making**
- Exploring **multi-agent interactions** within a structured framework

## **Getting Started**
Run the following cell to install the required libraries:



In [ ]:
# 🚀 Install necessary libraries for LangGraph-based AI workflows
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python langgraph wikipedia

# langchain_openai: Official integration with OpenAI models for LLM-based tasks
# langchain_core: Core framework for building and orchestrating LangChain applications
# langchain_community: Additional community-contributed integrations and utilities
# tavily-python: API client for web search, useful for Retrieval-Augmented Generation (RAG)
# langgraph: Framework for creating AI-driven stateful workflows and decision graphs


In [ ]:
# 🔑 Retrieve OpenAI API key securely from Colab's userdata storage
import os
from google.colab import userdata

openai_api_key = userdata.get('OpenAI_Key')  # Fetch the stored API key

# Ensure the key is set as an environment variable
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("✅ OpenAI API key successfully set.")
else:
    print("⚠️ OpenAI API key not found. Please store it using Colab's userdata feature.")


# 🤖 Using GPT-4o and GPT-3.5 in LangChain  

This section initializes **GPT-4o** and **GPT-3.5 Turbo** models using LangChain.  
It demonstrates how to:  
- Create a **human message** and send it as part of a conversation.  
- Invoke both models with **single text inputs** and **message lists**.  
- Print the responses for comparison.  

Run the code to see how different models respond to the same input.


In [ ]:
# 🤖 Initialize OpenAI Chat Models (GPT-4o & GPT-3.5)

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# Set up GPT-4o and GPT-3.5 Turbo with zero temperature for deterministic responses
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# 🗣️ Create a human message instance
msg = HumanMessage(content="Hello world", name="Lance")

# Store the message inside a list (models expect message sequences)
messages = [msg]

#  Invoke the GPT-4o model with a message list
response_4o = gpt4o_chat.invoke(messages)

#  Invoke the GPT-4o and GPT-3.5 models with a simple text prompt
response_4o_text = gpt4o_chat.invoke("hello world")
response_35_text = gpt35_chat.invoke("hello world")

# ✅ Display responses
print("GPT-4o Response:", response_4o)
print("GPT-4o (Text) Response:", response_4o_text)
print("GPT-3.5 Response:", response_35_text)


# 🔎 What is Tavily?  

[Tavily](https://tavily.com/) is a web search API that allows AI applications to retrieve real-time information from the internet.  
It is commonly used in *retrieval-augmented generation (RAG)* systems, where an LLM enhances responses by fetching *up-to-date* and relevant data from external sources.  

In this lab, Tavily will be used to perform web searches and integrate real-world information into LangGraph workflows.  


In [ ]:
# 🔑 Retrieve Tavily API key securely from Colab's userdata storage
tavily_api_key = userdata.get('Tavily_Key')  # Fetch the stored API key

# Ensure the key is set as an environment variable
if tavily_api_key:
    os.environ["TAVILY_API_KEY"] = tavily_api_key
    print("✅ Tavily API key successfully set.")
else:
    print("⚠️ Tavily API key not found. Please store it using Colab's userdata feature.")


In [ ]:
# 🔍 Using Tavily for Web Search in LangChain

from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize Tavily search tool with a maximum of 3 results
tavily_search = TavilySearchResults(max_results=3)

# Perform a web search query
query = "What is LangGraph?"
search_docs = tavily_search.invoke(query)

# ✅ Display the retrieved search results
print("🔹 Tavily Search Results for:", query)
print(search_docs)


# The Simplest Graph  

Let's build a simple graph with **three nodes** and **one conditional edge**. This structure allows an agent to make a decision at a branching point, directing the flow based on predefined conditions.  

<img src="https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dba5f465f6e9a2482ad935_simple-graph1.png" width="600"/>  

This example demonstrates how **LangGraph** enables structured decision-making while maintaining flexibility.  


## 🛠️ Understanding Graph Components  

LangGraph workflows consist of **three main components**: *State, Nodes, and Edges*. Each plays a key role in defining how information flows through the graph.  

---

## 🧠 State  

The **State** ([docs](https://langchain-ai.github.io/langgraph/concepts/low_level/#state)) acts as the **shared memory** of the graph, storing data that nodes can read and modify.  

- Every *node and edge* interacts with the state.  
- It is defined using `TypedDict` from Python’s `typing` module, which helps structure data with type hints.  
- Nodes modify the state by updating specific keys.  

---

## 🔵 Nodes  

A **Node** ([docs](https://langchain-ai.github.io/langgraph/concepts/low_level/#nodes)) is a simple **Python function** that processes the state.  

- The **first argument** of a node function is always the *state*.  
- Nodes can *read and modify* the state using keys like `state['graph_state']`.  
- By default, when a node returns a value, it updates the state and replaces the previous value ([reducers](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers) handle this).  

---

## 🔀 Edges  

An **Edge** ([docs](https://langchain-ai.github.io/langgraph/concepts/low_level/#edges)) connects nodes and controls how data moves between them.  

Two types of edges exist:  
- **Normal Edges** → Always transition from one node to the next (e.g., `node_1 → node_2`).  
- **Conditional Edges** ([docs](https://langchain-ai.github.io/langgraph/concepts/low_level/#conditional-edges)) → Decide the next node dynamically based on logic.  

Conditional edges *act like decision points*, determining the next step in the workflow.  


In [ ]:
from typing_extensions import TypedDict
from typing import Literal
import random

# 🎯 Define the State Schema
# The state acts as shared memory for the graph.
# It stores a key-value pair where 'graph_state' holds a string that evolves as nodes modify it.
class State(TypedDict):
    graph_state: str  # Tracks the sentence as it builds through the nodes

# 🔵 Define Nodes
# Each node modifies the 'graph_state' by appending its own text.
# Nodes simulate a simple sentence-building process.

def node_1(state: State) -> State:
    """Node 1 initializes the sentence with 'I am'."""
    print("--- Node 1 ---")
    return {"graph_state": state["graph_state"] + " I am"}

def node_2(state: State) -> State:
    """Node 2 completes the sentence with 'happy!'."""
    print("--- Node 2 ---")
    return {"graph_state": state["graph_state"] + " happy!"}

def node_3(state: State) -> State:
    """Node 3 completes the sentence with 'sad!'."""
    print("--- Node 3 ---")
    return {"graph_state": state["graph_state"] + " sad!"}

# 🔀 Define the Decision Function
# This function decides whether to send the state to Node 2 or Node 3.
# It randomly picks between the two, simulating an unpredictable emotional outcome.

def decide_mood(state: State) -> Literal["node_2", "node_3"]:
    """Randomly selects between Node 2 (happy) and Node 3 (sad)."""

    # Simulate a 50/50 decision
    if random.random() < 0.5:
        return "node_2"  # 50% chance to go to Node 2 (happy)
    return "node_3"  # 50% chance to go to Node 3 (sad)


## 🚀 Graph Construction  

Now, it's time to build the graph using the **components** defined earlier. The **StateGraph class** is used to create and manage the graph structure.  

### 🏗️ Steps to Build the Graph  
1. **Initialize the Graph** → Create a `StateGraph` using the `State` class.  
2. **Add Nodes and Edges** → Define how the graph flows.  
3. **Use Special Nodes**:  
   - **`START` Node** → Sends user input into the graph.  
   - **`END` Node** → Represents a terminal state.  
4. **Compile the Graph** → Ensures structural validity.  
5. **Visualize** → Convert it into a **Mermaid diagram** for better understanding.  

### 🔗 Reference Table  

| Concept       | Documentation Link |
|--------------|------------------|
| StateGraph Class | [StateGraph Docs](https://langchain-ai.github.io/langgraph/concepts/low_level/#stategraph) |
| START Node   | [START Node Docs](https://langchain-ai.github.io/langgraph/concepts/low_level/#start-node) |
| END Node     | [END Node Docs](https://langchain-ai.github.io/langgraph/concepts/low_level/#end-node) |
| Graph Compilation | [Compiling a Graph](https://langchain-ai.github.io/langgraph/concepts/low_level/#compiling-your-graph) |
| Mermaid Diagrams | [Mermaid Docs](https://github.com/mermaid-js/mermaid) |

This approach makes the graph more **structured, adaptable, and easy to debug**.  


In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# 🏗️ Initialize the Graph Builder
# The StateGraph is created using the State schema defined earlier.
builder = StateGraph(State)

# 🔵 Add Nodes
# Each node represents a step in the process and modifies the graph_state.
builder.add_node("node_1", node_1)  # Starts the sentence
builder.add_node("node_2", node_2)  # Appends "happy!"
builder.add_node("node_3", node_3)  # Appends "sad!"

# 🔗 Define Graph Flow (Edges)
# Edges define how nodes connect to each other.

# The START node sends input to "node_1"
builder.add_edge(START, "node_1")

# From "node_1", the next step is decided dynamically using the decide_mood function.
builder.add_conditional_edges("node_1", decide_mood)

# "node_2" (happy) and "node_3" (sad) both lead to the END node.
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# ✅ Compile the Graph
# This ensures that the structure is valid and ready for execution.
graph = builder.compile()

# 🖼️ Visualize the Graph
# Generates a Mermaid diagram to display the flow of nodes and edges.
display(Image(graph.get_graph().draw_mermaid_png()))


## Graph Invocation  

The compiled graph follows the [runnable](https://python.langchain.com/docs/concepts/runnables/) protocol, providing a standardized way to execute LangChain components. The `invoke` method is used to start execution, with an input dictionary like `{"graph_state": "Hi, this is Lance."}` setting the initial state. The graph begins at the `START` node and moves through the defined nodes (`node_1`, `node_2`, `node_3`) based on the control flow. A conditional edge determines whether the execution moves from `node_1` to `node_2` or `node_3`, following a 50/50 probability rule. Each node processes the current state, modifies the `graph_state` value, and returns the updated state. The execution continues along the directed edges until it reaches the `END` node, where the final graph state is returned.


In [ ]:
graph.invoke({"graph_state" : "Hi, this is Lance."})

# ✈️ Airport Security Screening Simulation (LangGraph Hands-On)

## 🛫 Scenario
In this exercise, we simulate an **airport security screening process** using **LangGraph**. Passengers arrive at security and are categorized into:
1. **TSA PreCheck (20%)** → Fast screening
2. **Regular Screening (80%)** → Standard security check
3. **Additional Screening (10% of Regular Passengers)** → Extra checks before proceeding to gates

Below is a **visual representation** of the screening process:

![Airport Security Flow](https://www.dropbox.com/scl/fi/o3ipy33svrcg64myu0u0s/AirPort_Security.png?rlkey=d6hc4bqdphzducnixba9ic2ai&dl=1)

---
## Task: Complete the Missing Code
Your goal is to **fill in the placeholders (`-------`)** to define functions and logic in the LangGraph framework.


In [ ]:
from typing_extensions import TypedDict
from typing import Literal
import random
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END


# 🎯 Define the State Schema
class AirportState(TypedDict):
    passenger_type: str  # Tracks whether passenger is Regular or TSA PreCheck

# 🔵 Define Nodes
def start_node(state: AirportState) -> AirportState:
    """Initial step where a passenger enters security."""
    print("🛫 Passenger arrives at security.")
    return state

def tsa_screening(state: -------) -> -------:
    """TSA PreCheck passengers go through expedited screening."""
    print("🟢 TSA PreCheck passenger goes through expedited screening.")
    return state

def regular_screening(state: -------) -> -------:
    """Regular passengers go through standard screening."""
    print("🔎 Regular passenger goes through standard screening.")
    return state

def additional_screening(state: -------) -> -------:
    """Additional screening for some regular passengers."""
    print("⚠️ Additional screening required for passenger.")
    return -------

def gates(state: -------) -> -------:
    """All passengers proceed to the boarding gates."""
    print("✅ Passenger cleared security. Proceeding to Gates!")
    return -------

# 🔀 Define the First Decision Function (TSA or Regular)
def assign_passenger_type(state: -------) -> Literal["-------", "-------"]:
    """Randomly assigns passengers to TSA PreCheck (20%) or Regular (80%)."""
    if random.random() < -------:
        print("🟢 Passenger assigned to TSA PreCheck.")
        return "tsa_screening"
    else:
        print("🔵 Passenger assigned to Regular Screening.")
        return "regular_screening"

# 🔀 Define the Second Decision Function (Regular -> Additional Screening or Gates)
def additional_screening_decision(state: -------) -> -------["-------", "-------"]:
    """10% of regular passengers go to additional screening, while 90% proceed to gates."""
    if random.random() < -------:
        print("⚠️ Passenger selected for additional screening.")
        return "-------"
    else:
        print("✅ Passenger cleared security after regular screening.")
        return "-------"

# 🏗️ Build the Graph
builder = StateGraph(-------)

# 🔵 Add Nodes
builder.-------("start_node", -------)
builder.add_node("tsa_screening", -------)
builder.add_node("regular_screening", -------)
builder.add_node("additional_screening", -------)
builder.add_node("gates", -------)

# 🔗 Define Graph Flow (Edges)
builder.-------(START, "-------")
builder.add_conditional_edges("start_node", -------)
builder.add_edge("tsa_screening", "-------")
builder.add_conditional_edges("regular_screening", -------)
builder.add_edge("additional_screening", "-------")
builder.add_edge("gates", END)

# ✅ Compile the Graph
graph = builder.-------

# 🖼️ Visualize the Graph
display(Image(graph.get_graph().-------))

# 🚀 Run the Graph
graph.-------({})


In [ ]:
# # 🔄 Test your code: Refresh this cell to see how the model works on different scenarios

graph.invoke({})

In [ ]:
from typing_extensions import TypedDict, Annotated
from typing import Literal
import random
import json
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langchain.schema import AIMessage, HumanMessage

# 🔑 Secure OpenAI API Key
OPENAI_API_KEY = userdata.get('OpenAI_Key')
llm = ChatOpenAI(model="gpt-4", temperature=0.7, api_key=OPENAI_API_KEY)

# 🎯 Define State Schema
class AirportState(TypedDict):
    passenger_type: str  # "TSA" or "Regular"
    luggage: Annotated[list, "merge"]  # Luggage details (set once)
    screening_notes: Annotated[list, "merge"]  # Screening decisions
    flagged: bool  # True if extra screening is needed

# 🏗️ Initialize Persistent Memory
memory_store = {"total_passengers": 0, "flagged_passengers": 0}

def create_passenger(state: AirportState) -> AirportState:
    """Creates a passenger and assigns luggage."""
    print("🛫 Passenger enters airport security.")

    # Generate luggage inside this function
    if "luggage" not in state or not state["luggage"]:

        # 🔀 Random chance to include dangerous items (10% of the time)
        if random.random() < 0.3:
            prompt = (
                "List 6-10 items in the passenger's luggage, including 1-2 dangerous items "
                "that might require extra screening. Format response as a JSON list."
                "(Example: [\"Laptop\", \"Water Bottle\", \"Pocket Knife\", \"Lighter\"])"
            )
            print("⚠️ Generating luggage with potential dangerous items...")
        else:
            prompt = (
                "List 6-10 common travel items in the passenger's luggage. "
                "Do NOT include dangerous items. Format response as a JSON list."
                "(Example: [\"Backpack\", \"Sunglasses\", \"Travel Pillow\", \"Shoes\"])"
            )

        # 🧠 Get response from LLM
        response = llm([HumanMessage(content=prompt)])

        try:
            items = json.loads(response.content)
            if isinstance(items, list):
                state["luggage"] = items
            else:
                raise ValueError("Invalid format received from LLM.")
        except json.JSONDecodeError:
            state["luggage"] = ["Unknown Item"]  # Fallback

    print(f"🎒 Luggage: {state['luggage']}")
    return state


def assign_lane(state: AirportState) -> Literal["tsa_entry", "regular_entry"]:
    """Assigns passenger to TSA or Regular screening."""
    if random.random() < 0.3:
        print("🟢 Passenger is TSA PreCheck.")
        state["passenger_type"] = "TSA"
        return "tsa_entry"
    else:
        print("🔵 Passenger is Regular Screening.")
        state["passenger_type"] = "Regular"
        return "regular_entry"

# ✅ TSA-ONLY SCREENING PATH
def tsa_entry(state: AirportState) -> AirportState:
    """TSA passengers enter the TSA checkpoint."""
    print("🟢 TSA passenger enters screening.")
    return state

def tsa_scan(state: AirportState) -> AirportState:
    """AI scans TSA passenger luggage for prohibited items."""
    return run_ai_screening(state, "TSA")

def tsa_decision(state: AirportState) -> Literal["tsa_extra", "tsa_clear"]:
    """Determines if TSA passenger needs extra screening."""
    return "tsa_extra" if state["flagged"] else "tsa_clear"

def tsa_extra(state: AirportState) -> AirportState:
    """TSA passengers flagged for extra screening."""
    print("⚠️ TSA passenger undergoing additional screening.")
    state.setdefault("screening_notes", []).append("TSA passenger sent to extra screening.")
    return state

def tsa_clear(state: AirportState) -> AirportState:
    """TSA passenger cleared to board."""
    return proceed_to_gates(state)

# ✅ REGULAR-ONLY SCREENING PATH
def regular_entry(state: AirportState) -> AirportState:
    """Regular passengers enter the standard security checkpoint."""
    print("🔎 Regular passenger enters screening.")
    return state

def regular_scan(state: AirportState) -> AirportState:
    """AI scans Regular passenger luggage for prohibited items."""
    return run_ai_screening(state, "Regular")

def regular_decision(state: AirportState) -> Literal["regular_extra", "regular_clear"]:
    """Determines if Regular passenger needs extra screening."""
    return "regular_extra" if state["flagged"] else "regular_clear"

def regular_extra(state: AirportState) -> AirportState:
    """Regular passengers flagged for extra screening."""
    print("⚠️ Regular passenger undergoing additional screening.")
    state.setdefault("screening_notes", []).append("Regular passenger sent to extra screening.")
    return state

def regular_clear(state: AirportState) -> AirportState:
    """Regular passenger cleared to board."""
    return proceed_to_gates(state)

# ✅ COMMON FINAL NODE
def proceed_to_gates(state: AirportState) -> AirportState:
    """Passengers cleared from both TSA & Regular screenings go to gates."""
    print("✅ Passenger cleared security. Proceeding to gates!")
    return state

# ✅ AI SCREENING FUNCTION (Shared)
def run_ai_screening(state: AirportState, passenger_type: str) -> AirportState:
    """AI determines if luggage requires additional screening."""
    global memory_store  # ✅ Track memory stats here

    prompt = f"Luggage: {state['luggage']}. Should this {passenger_type} passenger be flagged? Answer 'Yes' or 'No' with a short reason."
    response = llm([HumanMessage(content=prompt)])

    state["flagged"] = "yes" in response.content.lower()
    state.setdefault("screening_notes", []).append(response.content)

    # ✅ Update global memory stats
    memory_store["total_passengers"] += 1
    if state["flagged"]:
        memory_store["flagged_passengers"] += 1

    decision = "⚠️ Extra Screening Required!" if state["flagged"] else "✅ Cleared for Gates."
    print(f"🛃 AI Screening Decision: {decision} ({response.content})")

    return state

# 🏗️ Build Graph
builder = StateGraph(AirportState)

# 🔵 Add Nodes
builder.add_node("create_passenger", create_passenger)
builder.add_node("tsa_entry", tsa_entry)
builder.add_node("regular_entry", regular_entry)
builder.add_node("tsa_scan", tsa_scan)
builder.add_node("regular_scan", regular_scan)
builder.add_node("tsa_extra", tsa_extra)
builder.add_node("regular_extra", regular_extra)
builder.add_node("tsa_clear", tsa_clear)
builder.add_node("regular_clear", regular_clear)
builder.add_node("proceed_to_gates", proceed_to_gates)

# 🔗 Define Flow
builder.add_edge(START, "create_passenger")
builder.add_conditional_edges("create_passenger", assign_lane)

# TSA Path
builder.add_edge("tsa_entry", "tsa_scan")
builder.add_conditional_edges("tsa_scan", tsa_decision)
builder.add_edge("tsa_extra", "proceed_to_gates")
builder.add_edge("tsa_clear", "proceed_to_gates")

# Regular Path
builder.add_edge("regular_entry", "regular_scan")
builder.add_conditional_edges("regular_scan", regular_decision)
builder.add_edge("regular_extra", "proceed_to_gates")
builder.add_edge("regular_clear", "proceed_to_gates")

builder.add_edge("proceed_to_gates", END)

# ✅ Compile and Run
graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

# 🚀 Run for Multiple Passengers
for _ in range(6):  # Process 10 passengers in a single run
    print("\n🚀 Processing New Passenger 🚀")
    graph.invoke({})
    # ✅ Print Final Stats from Memory
    print(f"\n📈 FINAL STATS: {memory_store['total_passengers']} Total, {memory_store['flagged_passengers']} Flagged")


# 🤖 LangGraph with Human in the Loop 🔄

**Definition:**  
"Human in the Loop" refers to a workflow design where automated systems (such as AI and LangGraph) handle routine tasks, but key decisions are deferred to human judgment. This hybrid approach ensures that while the system processes data efficiently, a human can review and override decisions when necessary.

**Why It's Important:**  
- **✅ Quality Assurance:** Humans can catch subtleties and errors that automated processes may overlook.  
- **⚠️ Risk Mitigation:** Critical decisions benefit from human oversight, reducing potential risks from misclassification or errors.  
- **🔧 Adaptability:** Human feedback enables continuous improvement and adjustment of the automated system.

**Application in This Lab:**  
In this lab, the LangGraph system automatically screens passengers. If a passenger is flagged, the AI generates a 2-3 sentence comment describing potential risks. Then, a dropdown is presented for the user to decide whether to **"Pass"** or **"Reject"** the flagged passenger. This integration ensures that while automation speeds up the process, human insight remains central to final decision-making.


In [ ]:
from typing_extensions import TypedDict, Annotated
from typing import Literal
import random
import json
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langchain.schema import AIMessage, HumanMessage

# 🔑 Secure OpenAI API Key
OPENAI_API_KEY = userdata.get('OpenAI_Key')
llm = ChatOpenAI(model="gpt-4", temperature=0.7, api_key=OPENAI_API_KEY)

# 🎯 Define State Schema
class AirportState(TypedDict):
    passenger_type: str  # "TSA" or "Regular"
    luggage: Annotated[list, "merge"]  # Luggage details (set once)
    screening_notes: Annotated[list, "merge"]  # Screening decisions
    flagged: bool  # True if extra screening is needed

# 🏗️ Initialize Persistent Memory
memory_store = {"total_passengers": 0, "flagged_passengers": 0}

def create_passenger(state: AirportState) -> AirportState:
    """Creates a passenger and assigns luggage."""
    print("🛫 Passenger enters airport security.")

    # Generate luggage inside this function
    if "luggage" not in state or not state["luggage"]:
        # 🔀 Random chance to include dangerous items (10% of the time)
        if random.random() < 0.3:
            prompt = (
                "List 6-10 items in the passenger's luggage, including 1-2 dangerous items "
                "that might require extra screening. Format response as a JSON list."
                "(Example: [\"Laptop\", \"Water Bottle\", \"Pocket Knife\", \"Lighter\"])"
            )
            print("⚠️ Generating luggage with potential dangerous items...")
        else:
            prompt = (
                "List 6-10 common travel items in the passenger's luggage. "
                "Do NOT include dangerous items. Format response as a JSON list."
                "(Example: [\"Backpack\", \"Sunglasses\", \"Travel Pillow\", \"Shoes\"])"
            )

        # 🧠 Get response from LLM
        response = llm([HumanMessage(content=prompt)])

        try:
            items = json.loads(response.content)
            if isinstance(items, list):
                state["luggage"] = items
            else:
                raise ValueError("Invalid format received from LLM.")
        except json.JSONDecodeError:
            state["luggage"] = ["Unknown Item"]  # Fallback

    print(f"🎒 Luggage: {state['luggage']}")
    return state


def assign_lane(state: AirportState) -> Literal["tsa_entry", "regular_entry"]:
    """Assigns passenger to TSA or Regular screening."""
    if random.random() < 0.3:
        print("🟢 Passenger is TSA PreCheck.")
        state["passenger_type"] = "TSA"
        return "tsa_entry"
    else:
        print("🔵 Passenger is Regular Screening.")
        state["passenger_type"] = "Regular"
        return "regular_entry"

# ✅ TSA-ONLY SCREENING PATH
def tsa_entry(state: AirportState) -> AirportState:
    """TSA passengers enter the TSA checkpoint."""
    print("🟢 TSA passenger enters screening.")
    return state

def tsa_scan(state: AirportState) -> AirportState:
    """AI scans TSA passenger luggage for prohibited items."""
    return run_ai_screening(state, "TSA")

def tsa_decision(state: AirportState) -> Literal["tsa_extra", "tsa_clear"]:
    """Determines if TSA passenger needs extra screening."""
    return "tsa_extra" if state["flagged"] else "tsa_clear"

def tsa_extra(state: AirportState) -> AirportState:
    """TSA passengers flagged for extra screening."""
    print("⚠️ TSA passenger undergoing additional screening.")
    state.setdefault("screening_notes", []).append("TSA passenger sent to extra screening.")
    return state

def tsa_clear(state: AirportState) -> AirportState:
    """TSA passenger cleared to board."""
    return proceed_to_gates(state)

# ✅ REGULAR-ONLY SCREENING PATH
def regular_entry(state: AirportState) -> AirportState:
    """Regular passengers enter the standard security checkpoint."""
    print("🔎 Regular passenger enters screening.")
    return state

def regular_scan(state: AirportState) -> AirportState:
    """AI scans Regular passenger luggage for prohibited items."""
    return run_ai_screening(state, "Regular")

def regular_decision(state: AirportState) -> Literal["regular_extra", "regular_clear"]:
    """Determines if Regular passenger needs extra screening."""
    return "regular_extra" if state["flagged"] else "regular_clear"

def regular_extra(state: AirportState) -> AirportState:
    """Regular passengers flagged for extra screening."""
    print("⚠️ Regular passenger undergoing additional screening.")
    state.setdefault("screening_notes", []).append("Regular passenger sent to extra screening.")
    return state

def regular_clear(state: AirportState) -> AirportState:
    """Regular passenger cleared to board."""
    return proceed_to_gates(state)

# ✅ COMMON FINAL NODE
def proceed_to_gates(state: AirportState) -> AirportState:
    """Passengers cleared from both TSA & Regular screenings go to gates."""
    print("✅ Passenger cleared security. Proceeding to gates!")
    return state

# ✅ AI SCREENING FUNCTION (Shared with LLM providing comment & text input for decision)
def run_ai_screening(state: AirportState, passenger_type: str) -> AirportState:
    """AI determines if luggage requires additional screening. If flagged, provide a 2-3 sentence comment
    and prompt the user with a Y/N text input for decision before proceeding."""
    global memory_store  # ✅ Track memory stats here

    # Initial screening prompt
    prompt = f"Luggage: {state['luggage']}. Should this {passenger_type} passenger be flagged? Answer 'Yes' or 'No' with a short reason."
    response = llm([HumanMessage(content=prompt)])
    state["flagged"] = "yes" in response.content.lower()
    state.setdefault("screening_notes", []).append(response.content)

    # ✅ Update global memory stats
    memory_store["total_passengers"] += 1
    if state["flagged"]:
        memory_store["flagged_passengers"] += 1

        # +++ UPDATED: For flagged passengers, have the LLM provide a 2-3 sentence comment.
        comment_prompt = (
            f"Passenger Flagged: Luggage Items: {state['luggage']}. "
            "Please provide a brief, 2-3 sentence comment that explains the potential risks associated with these items."
        )
        comment_response = llm([HumanMessage(content=comment_prompt)])
        state.setdefault("screening_notes", []).append("Flagged Comment: " + comment_response.content)
        print(f"🛃 LLM Comment on Flagged Passenger: {comment_response.content}")

        # +++ UPDATED: Prompt the user with a text input for decision (Y/N).
        user_input = input("Flagged passenger. Enter Y to Pass or N to Reject: ").strip().lower()  # Y/N input
        while user_input not in ['y', 'n']:
            print("Invalid input. Please enter Y or N.")
            user_input = input("Flagged passenger. Enter Y to Pass or N to Reject: ").strip().lower()
        user_decision = "pass" if user_input == 'y' else "reject"
        state.setdefault("screening_notes", []).append("User Decision: " + user_decision)
        print(f"User Decision on flagged passenger: {user_decision}")

    decision = "⚠️ Extra Screening Required!" if state["flagged"] else "✅ Cleared for Gates."
    print(f"🛃 AI Screening Decision: {decision} ({response.content})")

    return state

# 🏗️ Build Graph
builder = StateGraph(AirportState)

# 🔵 Add Nodes
builder.add_node("create_passenger", create_passenger)
builder.add_node("tsa_entry", tsa_entry)
builder.add_node("regular_entry", regular_entry)
builder.add_node("tsa_scan", tsa_scan)
builder.add_node("regular_scan", regular_scan)
builder.add_node("tsa_extra", tsa_extra)
builder.add_node("regular_extra", regular_extra)
builder.add_node("tsa_clear", tsa_clear)
builder.add_node("regular_clear", regular_clear)
builder.add_node("proceed_to_gates", proceed_to_gates)

# 🔗 Define Flow
builder.add_edge(START, "create_passenger")
builder.add_conditional_edges("create_passenger", assign_lane)

# TSA Path
builder.add_edge("tsa_entry", "tsa_scan")
builder.add_conditional_edges("tsa_scan", tsa_decision)
builder.add_edge("tsa_extra", "proceed_to_gates")
builder.add_edge("tsa_clear", "proceed_to_gates")

# Regular Path
builder.add_edge("regular_entry", "regular_scan")
builder.add_conditional_edges("regular_scan", regular_decision)
builder.add_edge("regular_extra", "proceed_to_gates")
builder.add_edge("regular_clear", "proceed_to_gates")

builder.add_edge("proceed_to_gates", END)

# ✅ Compile and Run
graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

# 🚀 Run for Multiple Passengers
for _ in range(6):  # Process 6 passengers in a single run
    print("\n🚀 Processing New Passenger 🚀")
    graph.invoke({})
    # ✅ Print Final Stats from Memory
    print(f"\n📈 FINAL STATS: {memory_store['total_passengers']} Total, {memory_store['flagged_passengers']} Flagged")


# Tools in LangGraph

LangGraph is a powerful framework that models processes as state graphs, enabling you to build agentic systems with clearly defined states and transitions. Tools in LangGraph are external modules or functions that can be integrated into your agent to extend its capabilities—such as data retrieval, analysis, or interaction—without altering the core graph structure.

## How Tools Enhance LangGraph Agents

Integrating tools allows your agent to access real-time information or perform complex computations that are outside the scope of the state graph itself. For example, a search tool can fetch up-to-date information from the web, enriching the agent's context and supporting more informed decision-making.

## Adding Tavily to This Lab

In this lab, we added the Tavily search tool to provide external context when a passenger's luggage is flagged for potential risk. Tavily is used to perform a web search on the flagged items, supplying relevant information—such as recent news or safety alerts—that can help the user decide whether to pass or reject the passenger.


In [ ]:
from typing_extensions import TypedDict, Annotated
from typing import Literal
import random
import json
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langchain.schema import AIMessage, HumanMessage

# ^^^^^ UPDATED: Import Tavily search tool from langchain_community.tools.tavily_search
from langchain_community.tools.tavily_search import TavilySearchResults

# 🔑 Secure API Keys
OPENAI_API_KEY = userdata.get('OpenAI_Key')
llm = ChatOpenAI(model="gpt-4", temperature=0.7, api_key=OPENAI_API_KEY)

# ^^^^^ UPDATED: Initialize Tavily search tool with a maximum of 3 results and pass the API key from userdata.
tavily_search_tool = TavilySearchResults(max_results=3, tavily_api_key=userdata.get('Tavily_Key'))

# 🎯 Define State Schema
class AirportState(TypedDict):
    passenger_type: str         # "TSA" or "Regular"
    luggage: Annotated[list, "merge"]         # Luggage details (set once)
    screening_notes: Annotated[list, "merge"] # Screening decisions
    flagged: bool               # True if extra screening is needed

# 🏗️ Initialize Persistent Memory
memory_store = {"total_passengers": 0, "flagged_passengers": 0}

def create_passenger(state: AirportState) -> AirportState:
    """Creates a passenger and assigns luggage."""
    print("🛫 Passenger enters airport security.")
    if "luggage" not in state or not state["luggage"]:
        if random.random() < 0.5:
            # ^^^^^ UPDATED: Mention dangerous or unusual items in the prompt.
            prompt = (
                "List 6-10 items in the passenger's luggage, including 1-2 dangerous or unusual items "
                "that might require extra screening. Format response as a JSON list. "
                "(Example: [\"Laptop\", \"Water Bottle\", \"Unknown Device\", \"Pocket Knife\"])"
            )
            print("⚠️ Generating luggage with potential dangerous/unusual items...")
        else:
            prompt = (
                "List 6-10 common travel items in the passenger's luggage. "
                "Do NOT include dangerous items. Format response as a JSON list. "
                "(Example: [\"Backpack\", \"Sunglasses\", \"Travel Pillow\", \"Shoes\"])"
            )
        response = llm([HumanMessage(content=prompt)])
        try:
            items = json.loads(response.content)
            if isinstance(items, list):
                state["luggage"] = items
            else:
                raise ValueError("Invalid format received from LLM.")
        except json.JSONDecodeError:
            state["luggage"] = ["Unknown Item"]
    print(f"🎒 Luggage: {state['luggage']}")
    return state

def assign_lane(state: AirportState) -> Literal["tsa_entry", "regular_entry"]:
    if random.random() < 0.3:
        print("🟢 Passenger is TSA PreCheck.")
        state["passenger_type"] = "TSA"
        return "tsa_entry"
    else:
        print("🔵 Passenger is Regular Screening.")
        state["passenger_type"] = "Regular"
        return "regular_entry"

def tsa_entry(state: AirportState) -> AirportState:
    print("🟢 TSA passenger enters screening.")
    return state

def tsa_scan(state: AirportState) -> AirportState:
    return run_ai_screening(state, "TSA")

def tsa_decision(state: AirportState) -> Literal["tsa_extra", "tsa_clear"]:
    return "tsa_extra" if state["flagged"] else "tsa_clear"

def tsa_extra(state: AirportState) -> AirportState:
    print("⚠️ TSA passenger undergoing additional screening.")
    state.setdefault("screening_notes", []).append("TSA passenger sent to extra screening.")
    return state

def tsa_clear(state: AirportState) -> AirportState:
    return proceed_to_gates(state)

def regular_entry(state: AirportState) -> AirportState:
    print("🔎 Regular passenger enters screening.")
    return state

def regular_scan(state: AirportState) -> AirportState:
    return run_ai_screening(state, "Regular")

def regular_decision(state: AirportState) -> Literal["regular_extra", "regular_clear"]:
    return "regular_extra" if state["flagged"] else "regular_clear"

def regular_extra(state: AirportState) -> AirportState:
    print("⚠️ Regular passenger undergoing additional screening.")
    state.setdefault("screening_notes", []).append("Regular passenger sent to extra screening.")
    return state

def regular_clear(state: AirportState) -> AirportState:
    return proceed_to_gates(state)

def proceed_to_gates(state: AirportState) -> AirportState:
    print("✅ Passenger cleared security. Proceeding to gates!")
    return state

def run_ai_screening(state: AirportState, passenger_type: str) -> AirportState:
    global memory_store
    prompt = f"Luggage: {state['luggage']}. Should this {passenger_type} passenger be flagged? Answer 'Yes' or 'No' with a short reason."
    response = llm([HumanMessage(content=prompt)])
    state["flagged"] = "yes" in response.content.lower()
    state.setdefault("screening_notes", []).append(response.content)
    memory_store["total_passengers"] += 1
    if state["flagged"]:
        memory_store["flagged_passengers"] += 1

        # ^^^^^ UPDATED: LLM provides a 2-3 sentence comment on risks.
        comment_prompt = (
            f"Passenger Flagged: Luggage Items: {state['luggage']}. "
            "Please provide a brief, 2-3 sentence comment that explains the potential risks associated with these items."
        )
        comment_response = llm([HumanMessage(content=comment_prompt)])
        state.setdefault("screening_notes", []).append("Flagged Comment: " + comment_response.content)
        print(f"🛃 LLM Comment on Flagged Passenger: {comment_response.content}")

        # ^^^^^ UPDATED: Call the Tavily search tool to get external information.
        search_query = " ".join(state["luggage"]) + " dangerous unusual security"
        tavily_result = tavily_search_tool.invoke(search_query)
        # ^^^^^ UPDATED: Convert each element to string if needed before joining.
        tavily_str = ", ".join([str(item) for item in tavily_result]) if isinstance(tavily_result, list) else str(tavily_result)
        state.setdefault("screening_notes", []).append("Tavily Search Result: " + tavily_str)
        print(f"🔍 Tavily Search Result: {tavily_str}")

        # ^^^^^ UPDATED: Prompt the user with a text input (Y/N) for decision.
        user_input = input("Flagged passenger. Enter Y to Pass or N to Reject: ").strip().lower()
        while user_input not in ['y', 'n']:
            print("Invalid input. Please enter Y or N.")
            user_input = input("Flagged passenger. Enter Y to Pass or N to Reject: ").strip().lower()
        user_decision = "pass" if user_input == 'y' else "reject"
        state.setdefault("screening_notes", []).append("User Decision: " + user_decision)
        print(f"User Decision on flagged passenger: {user_decision}")

    decision = "⚠️ Extra Screening Required!" if state["flagged"] else "✅ Cleared for Gates."
    print(f"🛃 AI Screening Decision: {decision} ({response.content})")
    return state

builder = StateGraph(AirportState)
builder.add_node("create_passenger", create_passenger)
builder.add_node("tsa_entry", tsa_entry)
builder.add_node("regular_entry", regular_entry)
builder.add_node("tsa_scan", tsa_scan)
builder.add_node("regular_scan", regular_scan)
builder.add_node("tsa_extra", tsa_extra)
builder.add_node("regular_extra", regular_extra)
builder.add_node("tsa_clear", tsa_clear)
builder.add_node("regular_clear", regular_clear)
builder.add_node("proceed_to_gates", proceed_to_gates)
builder.add_edge(START, "create_passenger")
builder.add_conditional_edges("create_passenger", assign_lane)
builder.add_edge("tsa_entry", "tsa_scan")
builder.add_conditional_edges("tsa_scan", tsa_decision)
builder.add_edge("tsa_extra", "proceed_to_gates")
builder.add_edge("tsa_clear", "proceed_to_gates")
builder.add_edge("regular_entry", "regular_scan")
builder.add_conditional_edges("regular_scan", regular_decision)
builder.add_edge("regular_extra", "proceed_to_gates")
builder.add_edge("regular_clear", "proceed_to_gates")
builder.add_edge("proceed_to_gates", END)
graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

for _ in range(6):
    print("\n🚀 Processing New Passenger 🚀")
    graph.invoke({})
    print(f"\n📈 FINAL STATS: {memory_store['total_passengers']} Total, {memory_store['flagged_passengers']} Flagged")
